#Relatório e Projeto de COCADA

A premissa do meu trabalho é coletar os dados de avaliações de filmes de um perfil da plataforma Letterboxd que for colocado no programa, ou então coletar avaliações manualmente, caso você não tenha um perfil, e com base nessas avaliações montar a equação Ax = b, sendo a matriz A com linhas sendo filmes e colunas sendo gêneros, o vetor b sendo a avaliação do usuário para cada filme, e x resultará no quanto o usuário gosta de cada gênero relativo a sua avaliações, pesando também a avaliação geral do mesmo, o que será mostrado na tela. Após isso, essa matriz x será usada para recomendar 4 filmes que o usuário ainda não avaliou, e que tenham seu rating geral maior que 4.0, e que ele possivelmente gostará utilizando o método de mínimos quadrados. Utilizarei um dataset contendo filmes e seus respectivos gêneros para isso, além dos dados de avaliações do usuário coletados do Letterboxd, utilizando a técnica de Web Scraping.

#Célula 0
Esta célula instala as bibliotecas necessárias para o projeto: `requests`, `beautifulsoup4`, `pandas`, `numpy`, `matplotlib` e `scipy`.
Elas são fundamentais para realizar o scraping dos dados da web, manipulação de tabelas, operações numéricas vetoriais, visualização gráfica e resolução de sistemas de equações com restrições.
Pesquisei a função de cada uma dessas bibliotecas para tentar aplicá-las corretamente neste projeto.

In [ ]:
# Célula 0: Instalação das dependencias

!pip install requests beautifulsoup4 pandas numpy matplotlib scipy

#Célula 1
Aqui fiz os imports necessários para rodar o sistema. São trazidas funções para:
- acessar páginas da internet (`requests`);
- extrair informações do HTML (`BeautifulSoup`);
- trabalhar com dados tabulares (`pandas`);
- realizar operações matriciais e numéricas (`numpy`);
- interpretar strings como listas (`ast`);
- pausar requisições para não sobrecarregar o servidor (`time`);
- exibir gráficos com os resultados (`matplotlib`);
- resolver sistemas lineares com restrições, usando mínimos quadrados (`scipy.optimize.lsq_linear`).

Cada biblioteca foi selecionada com base em pesquisas.

In [ ]:
# Célula 1: Imports e configurações iniciais

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ast
import time
import matplotlib.pyplot as plt
from scipy.optimize import lsq_linear


# Cabeçalhos para simular um navegador real
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/91.0.4472.124 Safari/537.36'
}

#Célula 2
Esta função coleta as avaliações de filmes de um usuário do Letterboxd, página por página.
Foi necessário pegar a estrutura do HTML do site e utilizar o BeautifulSoup para acessar os dados corretamente.
Ela retorna um DataFrame com os títulos dos filmes e suas respectivas notas (de 0.5 a 5).
Esse processo envolve conceitos de **web scraping**, extração de dados estruturados e automação de coleta de dados.
Eu peguei mais funções e coisas prontas para essa parte, já que não está relacionada com a matéria em si.

In [ ]:
# Célula 2: Scraping das páginas de avaliações

def get_user_ratings(username, pause=1.0):
    ratings, page = [], 1
    while True:
        url = f"https://letterboxd.com/{username}/films/by/rated-date/page/{page}/"
        try:
            resp = requests.get(url, headers=headers, timeout=15)
            resp.raise_for_status()
        except requests.RequestException:
            break
        html = resp.text.lower()
        if page == 1:
            if 'this profile is private' in html:
                print(f"Perfil '{username}' é privado.")
                return pd.DataFrame(columns=['title','rating'])
            if 'we can’t find the page' in html:
                print(f"Perfil '{username}' não encontrado.")
                return pd.DataFrame(columns=['title','rating'])
        soup = BeautifulSoup(resp.text, 'html.parser')
        posters = soup.select('li.poster-container')
        if not posters:
            break
        for li in posters:
            img = li.find('img', class_='image')
            span = li.find('span', class_='rating')
            if img and span:
                title = img.get('alt','').strip()
                tag = [c for c in span.get('class',[]) if c.startswith('rated-')]
                if tag:
                    ratings.append({'title': title,
                                    'rating': int(tag[0].split('-')[1]) / 2.0})
        if not (soup.select_one('a.next') or soup.select_one('li.paginate-page a[rel="next"]')):
            break
        page += 1
        time.sleep(pause)
    print(f"Total de {len(ratings)} avaliações coletadas de '{username}'")
    return pd.DataFrame(ratings)

#Célula 3
Esta função permite que o usuário insira manualmente os títulos dos filmes e suas respectivas notas, caso não deseje usar os dados do Letterboxd, ou não tenha. Essa abordagem é importante para garantir que o sistema funcione mesmo sem conexão à internet ou para testes rápidos.
A entrada é validada para garantir que as notas estejam no intervalo permitido (0.5 a 5.0). A manipulação do DataFrame ao final é feita com `pandas`.

In [ ]:
# Célula 3: Entrada manual de avaliações

def get_manual_ratings():
    entries = []
    while True:
        title = input("Título (ou 'fim'): ").strip()
        if title.lower()=='fim': break
        try:
            rating = float(input("Nota (0.5-5.0): ").replace(',','.'))
            if 0.5 <= rating <= 5.0:
                entries.append({'title':title, 'rating':rating})
            else:
                print("Fora do intervalo.")
        except ValueError:
            print("Inválido.")
    return pd.DataFrame(entries)


#Célula 4
Esta célula é responsável por carregar o banco de dados local com informações sobre filmes e seus respectivos gêneros. São feitas as seguintes etapas:
- Leitura do arquivo CSV contendo os filmes, com `pandas.read_csv()`;
- Renomeação das colunas para padronização;
- Tratamento de valores ausentes em gêneros (substituindo por listas vazias);
- Conversão das strings representando listas de gêneros usando `ast.literal_eval`.

Esse tipo de pré-processamento foi ideal para garantir que os dados estejam em formato adequado para análises posteriores, como operações matriciais e filtragens.

In [ ]:
# Célula 4: Carrega o dataset

def load_local_movie_database(filepath='Movie_Data_File.csv'):
    df = pd.read_csv(filepath)
    df = df.rename(columns={
        'Film_title':'title',
        'Genres':'genres',
        'Average_rating':'rating'})
    df['genres'] = df['genres'].fillna('[]')
    df['genres'] = df['genres'].apply(lambda s: ast.literal_eval(s) if isinstance(s,str) else [])
    return df[['title','genres','rating']]


#Célula 5
Esta célula é o núcleo matemático do projeto, onde aplicamos álgebra linear na forma da equação Ax = b:
- `A` é a matriz de gêneros (cada linha representa um filme e cada coluna, um gênero; valores binários indicam se o filme pertence a cada gênero);
- `b` é o vetor de avaliações atribuídas pelo usuário, ponderadas com a média global do filme, para melhorar a robustez do modelo;
- As linhas da matriz A são normalizadas para evitar que filmes com muitos gêneros tenham mais peso na equação.

Este processo se alinha muito a mínimos quadrados. Tive que aplicar também, juntamente com os pesos, uma média ponderada da nota do usuário com o rating global para cada filme, sendo de 70% e 30% respectivamente, assim dando um resultado mais preciso.

In [ ]:
# Célula 5: Montar A e b (corrigido com pesos e normalização)

def build_matrices(user_df, movie_db_df):
    df = pd.merge(user_df, movie_db_df, on='title', how='inner', suffixes=('_user','_global'))
    if df.empty:
        raise RuntimeError("Nenhum filme avaliado no banco local.")
    genres = sorted({g for lst in df['genres'] for g in lst})
    A = np.zeros((len(df), len(genres)))
    b = np.zeros(len(df))
    max_rating = movie_db_df['rating'].max()
    for i,row in enumerate(df.itertuples(index=False)):
        for g in row.genres:
            A[i, genres.index(g)] = 1
        norm = np.linalg.norm(A[i]) or 1
        A[i] /= norm  # Normaliza linha para evitar somas grandes
        global_norm = (row.rating_global / max_rating) * 5
        b[i] = 0.7 * row.rating_user + 0.3 * global_norm  # Peso maior na nota do usuário
    return A,b,genres,df['title'].tolist()

#Célula 6
Aqui usamos `np.linalg.lstsq()` para resolver o sistema Ax = b por mínimos quadrados da forma que minimiza o erro quadrático entre as avaliações previstas e as reais.
O vetor `x` resultante representa os pesos do usuário para cada gênero — ou seja, o quanto ele gosta de cada um deles.

In [ ]:
# Célula 6: Resolve X

def solve_genre_weights(A, b):
    lb = np.full(A.shape[1], 0.5)
    ub = np.full(A.shape[1], 5.0)
    res = lsq_linear(A, b, bounds=(lb, ub))
    return res.x


#Célula 7
Esta função cria um gráfico de barras que mostra os pesos (preferências) do usuário para cada gênero fornecido. Ela usa matplotlib para plotar os valores, ajustando os rótulos dos gêneros para melhor visualização. O gráfico facilita a visualização das preferências categóricas do usuário.

In [ ]:
# Célula 7: Plot dos pesos por gênero

def plot_genre_weights(genres, x):
    plt.figure()
    plt.bar(genres, x)
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Peso')
    plt.title('Preferências do usuário por gênero')
    plt.tight_layout()
    plt.show()


#Célula 8
Com base no vetor de pesos `x`, calculamos o quão bem cada filme não assistido combina com os gostos do usuário. O processo envolve:
- Filtrar filmes que o usuário ainda não viu e com média global >= 4.0;
- Criar uma matriz `M` semelhante à `A`, com gêneros dos filmes candidatos;
- Normalizar as linhas de `M` e multiplicar por `x` para obter uma nota prevista (quanto mais alta, maior a chance do usuário gostar);
- Ordenar os resultados e retornar os 4 melhores.

Esse procedimento usa o produto escalar para medir afinidade entre vetores.


In [ ]:
# Célula 8: Recomendações usando raw_score definido abaixo

def recommend_movies(x, genres, movie_db_df, seen, top_n=4, debug=False):
    # Seleciona os índices dos 3 gêneros favoritos (maiores pesos)
    top3_idx = np.argsort(x)[-3:]
    top3_genres = [genres[i] for i in top3_idx]

    # Filtra candidatos não vistos e com rating > 4
    cand = movie_db_df[
    (~movie_db_df['title'].isin(seen)) & (movie_db_df['rating'] >= 4.0)
].copy().reset_index(drop=True)


    # Matriz considerando apenas os 3 gêneros favoritos
    M = np.zeros((len(cand), len(top3_genres)))

    for i, row in enumerate(cand.itertuples(index=False)):
        for j, g in enumerate(top3_genres):
            if g in row.genres:
                M[i, j] = 1

    # Cálculo do raw_score: média aritmética dos pesos apenas dos 3 gêneros favoritos
    counts = M.sum(axis=1)
    counts[counts == 0] = 1
    raw_scores = (M * x[top3_idx]).sum(axis=1) / counts

    pred = np.clip(raw_scores, 0.5, 5.0)
    cand['predicted'] = pred

    if debug:
        detailed = []
        for i, title in enumerate(cand['title']):
            active = [g for g in cand.at[i, 'genres'] if g in top3_genres]
            detailed.append({'title': title, 'raw_score': raw_scores[i], 'genres': active})
        debug_df = pd.DataFrame(detailed).sort_values('raw_score', ascending=False).head(top_n)

        result = cand.sort_values('predicted', ascending=False).head(top_n).copy()
    result['genres'] = result['title'].map(
        dict(zip(movie_db_df['title'], movie_db_df['genres']))
    )
    result['ranking geral'] = np.arange(1, len(result) + 1)
    return result[['ranking geral', 'title', 'genres']]


#Célula 9
Exibi, em formato de DataFrame, a matriz `A` com os gêneros de cada filme e a respectiva nota ponderada `b`. Isso permite:
- Verificar quais filmes foram usados para montar a matriz;
- Entender como cada filme contribui para a formação do perfil do usuário.

In [ ]:
# Célula 9: Visualização dos dados usados na equação Ax = b

def show_user_data_matrix(A, b, genres, titles):
    df = pd.DataFrame(A, columns=genres)
    df['title'] = titles
    df['b (nota esperada)'] = b
    return df.sort_values(
        'b (nota esperada)',
        ascending=False
    ).reset_index(drop=True)

#Célula 10
Este é o ponto de entrada do programa:
- Solicita o nome de usuário ou ativa o modo de entrada manual;
- Coleta as notas e os filmes assistidos;
- Carrega o banco local e monta a matriz A e vetor b;
- Resolve o sistema por mínimos quadrados e imprime os pesos dos gêneros;
- Exibe os dados usados na criação do perfil e as recomendações finais.

Aqui a integração de todas as partes anteriores ocorre e elas são de fato executadas.

In [ ]:
# Célula 10: Main

if __name__ == '__main__':
    user = input("User Letterboxd (vazio -> manual): ")
    df_user = (
        get_user_ratings(user) if user.strip() else get_manual_ratings()
    )
    db = load_local_movie_database()
    A, b, gens, seen = build_matrices(df_user, db)
    x = solve_genre_weights(A, b)

    print("\nPesos:")
    for g, w in zip(gens, x):
        print(f" {g}: {w:.2f}")

    plot_genre_weights(gens, x)

    print("\nAlguns filmes usados para o peso e suas notas relativas:")
    print(
        show_user_data_matrix(A, b, gens, seen)[
            ['title', 'b (nota esperada)']
        ].head(10).to_string(index=False)
    )

    recs = recommend_movies(x, gens, db, seen, debug=True)
    print("🎯 Recomendações Personalizadas 🎬\n")
    for _, row in recs.iterrows():
      print(f"🏆 #{row['ranking geral']} — {row['title']} 🎞️")
      print(f"   Gêneros: {', '.join(row['genres'])}\n")




#**Considerações finais:**



Minha ideia para um sistema de profiling e recomendação de filmes baseado no site Letterboxd foi inspirada no meu projeto de ALA do período 24.2, que era apenas um sistema de recomendações de filmes baseado em avaliações dadas na hora, e apesar de não ter reutilizado nenhum código do mesmo, minha ideia foi inspirada dele.

Ao longo da execução do projeto fui notando que a complexidade do que eu tinha em mente era bem maior do que a esperada, principalmente devido à técnica de Web Scraping e para lidar com os dados na matriz, então tive que improvisar em diversas áreas do código em python, envolveu muita pesquisa, porém no fim consegui fazer com que tudo funcionasse corretamente.

Além disso, também encontrei diversos obstáculos para escolher o dataset, a primor eu havia conseguido uma chave para uma API pública após fazer a solicitação para o site TMDb, porém os gêneros dos filmes desse dataset estavam relativamente errados, ou pelo menos forçados, por exemplo, um dos gêneros de Interestelar era Western, e sci-fi não estava na lista do mesmo, o que acabava gerando resultados muito estranhos nas recomendações, gostar de Interestelar era sinônimo de gostar de Django Livre, que são filmes bem diferentes um do outro.

Depois desse dataset que considerei falho, busquei outro e acabei encontrando um da plataforma Kaggle, e que de fato parecia estar mais correto, apesar de algumas inconsistências, porém tive que mudar completamente o código da célula para utilizar nas matrizes os filmes e gêneros desse novo dataset, o que foi um trabalho bem complexo e inesperado.

#Referências:

*   Dataset utilizado: https://www.kaggle.com/datasets/ky1338/10000-movies-letterboxd-data

*   Referência para o Web Scraping: https://github.com/nmcassa/letterboxdpy

*   Funções desconhecidas por mim foram tiradas somente de pesquisas em diversas outras fontes variadas e vendo suas utilizações práticas, sem muito foco dado a elas.